### 청약 매물 기사 크롤링 함수
- 내용에 '청약' 포함 기사만 크롤링
- 한 매물당 기사 크롤링 최대 개수는 10개로 제한
********************************************
- 매물 하나만 크롤링하고 싶다 -> 1~3번 수행
- 매물 데이터셋이 있고 크롤링하고 싶다 -> 4번 수행 
********************************************
- 4번 코드는 다음과 같이 작동함
    1. 아파트 이름과 "청약" 키워드를 조합하여 네이버 뉴스를 검색
    2. 검색 결과에서 "네이버 뉴스" 링크가 있는 기사만 선별
    3. 선별된 기사의 전문(full content)을 크롤링
    4. 크롤링된 기사 내용에 "청약" 키워드가 포함된 경우에만 저장
    5. 각 아파트당 최대 10개의 기사를 수집
    6. 이 과정을 모든 수도권 아파트에 대해 반복
    7. 최종 결과를 CSV 파일로 저장

In [17]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

1. 검색 URL 생성 함수 (아파트명 + 청약)

In [7]:
import urllib.parse

def generate_news_url(apartment_name: str) -> str:
    """아파트명 + 청약 검색 URL 생성"""
    base_url = "https://search.naver.com/search.naver"
    query = f'"{apartment_name}" 청약'  # 쌍따옴표 포함 검색
    
    params = {
        "where": "news",
        "query": query,
        "sm": "tab_opt",
        "sort": "1"  # 최신순 정렬
    }
    
    return base_url + "?" + urllib.parse.urlencode(params, quote_via=urllib.parse.quote)

2. 네이버 뉴스 전용 크롤링 함수

In [32]:
import requests
from bs4 import BeautifulSoup

def crawl_naver_news(url: str, max_articles: int = 10) -> list:
    """청약 관련 기사만 필터링하는 네이버 뉴스 크롤러"""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # 검색 결과 페이지 파싱
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 네이버 뉴스 링크 필터링
    news_links = [
        a['href'] for a in soup.select('a.info')
        if '네이버뉴스' in a.text and 'news.naver.com' in a['href']
    ][:max_articles]
    
    # 기사 본문 추출 + 청약 필터링
    articles = []
    for article_url in news_links:
        try:
            article_res = requests.get(article_url, headers=headers, timeout=5)
            article_soup = BeautifulSoup(article_res.text, 'html.parser')
            
            # 본문 추출
            content = article_soup.select_one('#newsct_article, #dic_area').get_text(strip=True)
            
            # 청약 키워드 필터링
            if '청약' not in content:
                print(f"제외: {article_url} (청약 미포함)")
                continue
                
            # 필수 정보 추출
            articles.append({
                "title": article_soup.select_one('#title_area').get_text(strip=True),
                "date": article_soup.select_one('.media_end_head_info_datestamp_time')['data-date-time'],
                "content": content,
                "url": article_url
            })
            
        except Exception as e:
            print(f"기사 처리 실패 ({article_url}): {str(e)}")
    
    return articles

3. 통합 실행

In [33]:
if __name__ == "__main__":
    # 아파트명 입력
    apartment_name = "래미안 원베일리"
    
    # 1단계: 검색 URL 생성
    search_url = generate_news_url(apartment_name)
    print(f"생성된 검색 URL: {search_url}")
    
    # 2단계: 네이버 뉴스 기사 크롤링
    news_data = crawl_naver_news(search_url)
    
    # 결과 출력
    print(f"\n{apartment_name} 관련 청약 뉴스 {len(news_data)}건")
    for idx, article in enumerate(news_data, 1):
        print(f"\n[{idx}] {article['title']}")
        print(f"작성일: {article['date']}")
        print(f"요약: {article['content']}")
        print(f"URL: {article['url']}")

생성된 검색 URL: https://search.naver.com/search.naver?where=news&query=%EB%9E%98%EB%AF%B8%EC%95%88+%EC%9B%90%EB%B2%A0%EC%9D%BC%EB%A6%AC+%EC%B2%AD%EC%95%BD&sm=tab_jum&sort=1


Exception ignored in: <function tqdm.__del__ at 0x1076d1f80>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/Yeeun_Han/lib/python3.11/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/opt/anaconda3/envs/Yeeun_Han/lib/python3.11/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'



래미안 원베일리 관련 청약 뉴스 9건

[1] 아이유·송중기 사는 `그집`, 올해 가장 비싼 아파트…국내 최초 공시가 200억
작성일: 2025-03-13 12:12:08
요약: 가수 아이유와 배우 송중기.<이담엔터테인먼트 제공, 연합뉴스>에테르노 청담.<에테르노 청담 홍페이지, 연합뉴스>전국에서 공시가격이 가장 비싼 아파트는 서울 강남구 청담동 '에테르노 청담'인 것으로 조사됐다. 가장 비싼 아파트 타이틀은 4년 연속 에테르노 청담과 300m 떨어져 있는 '더펜트하우스 청담'이 차지하고 있었으나 이번에 1위 자리를 넘겼다.13일 국토교통부가 공개한 올해 공동주택 공시가격(안)에 따르면 에테르노 청담 전용면적 464.11㎡의 올해 공시가격은 200억6000만원으로 전국에서 가장 높았다. 지난해보다 공시가격이 72억원이나 뛰었다.에테르노 청담은 2023년 말 준공 승인을 받은 청담동 한강변 아파트로, 프리츠커상 수상자인 라파엘 모네오가 설계했다. 분양가 상한제와 공개 청약 규제를 피할 수 있는 29가구 규모다.이 아파트는 지난해 공시가격이 산정되자마자 2위에 오르더니, 올해는 1위로 올라섰다. 현재 전용면적 244㎡가 220억원에, 공급면적 565.5㎡·전용면적 273㎡의 듀플렉스(복층 구조)는 320억원에 매물로 나와 있다.공시가격 상위 2위는 청담동 더펜트하우스청담으로, 전용면적 407.71㎡의 올해 공시가격이 172억1000만원이다. 1년 새 8억1000만원이나 올랐다.호텔 엘루이 부지에 지어진 더펜트하우스 청담은 2020년 8월 입주했다. 입지 여건은 비슷한데 에테르노 청담의 집 크기가 더 크고, 신축이라는 점이 올해 공시가격 역전에 영향을 미친 것으로 보인다. 공시가격 상위 1·2위 아파트의 시공사는 모두 현대건설이다.상위 3위는 지난해와 같은 서울 용산구 한남동 나인원한남이다. 전용면적 244.72㎡가 163억원으로 지난해보다 56억3000만원 올랐다. 4위는 용산구 한남동 한남더힐 244.75㎡로 118억6000만원이다.올해는 반포 래미안원베일리가

4. 수도권 매물만 기사 크롤링

In [ ]:
import pandas as pd
import time
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import urllib.parse
import re

def generate_news_url(apartment_name):
    base_url = "https://search.naver.com/search.naver"
    # 괄호와 그 안의 내용 제거
    apartment_name = re.sub(r'\([^)]*\)', '', apartment_name).strip()
    query = f'"{apartment_name}" 청약'
    params = {
        "where": "news",
        "query": query,
        "sm": "tab_opt",
        "sort": "1"
    }
    return base_url + "?" + urllib.parse.urlencode(params)

def crawl_naver_news(url, max_articles=10):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    articles = []
    page = 1  # 페이지 번호 초기화
    
    while len(articles) < max_articles:  # 원하는 최대 기사 수에 도달할 때까지 반복
        try:
            # 페이지 URL 생성 (페이지 번호 적용)
            paged_url = f"{url}&start={(page - 1) * 10 + 1}"
            response = requests.get(paged_url, headers=headers)
            response.raise_for_status()  # HTTP 에러 확인

            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 기사 영역 선택
            news_areas = soup.select(".news_area")
            
            # 기사 영역이 없으면 종료
            if not news_areas:
                print("더 이상 기사 영역이 없습니다.")
                break

            for item in news_areas:
                # "네이버 뉴스" 버튼이 있는지 확인
                naver_news_link = item.select_one("a[href*='news.naver.com']")
                if not naver_news_link:
                    continue  # "네이버 뉴스" 링크가 없으면 다음 기사로 건너뜀
                
                title = item.select_one(".news_tit").text
                news_url = naver_news_link['href']  # 네이버 뉴스 URL 사용

                # 기사 본문 크롤링 함수 호출
                full_content = crawl_article_content(news_url, headers)

                # 키워드 필터링
                if "청약" in full_content:  # 크롤링된 전체 내용에서 "청약" 키워드 확인
                    articles.append({"title": title, "content": full_content, "url": news_url})
            
            # 다음 페이지로 이동
            page += 1
            time.sleep(3)  # 페이지 요청 간 3초 대기

        except requests.exceptions.RequestException as e:
            print(f"요청 에러 발생: {e}")
            break  # 요청 에러 발생 시 크롤링 중단
        except Exception as e:
            print(f"파싱 에러 발생: {e}")
            break  # 파싱 에러 발생 시 크롤링 중단

    return articles

def crawl_article_content(news_url, headers):
    """기사 URL을 받아서 전체 내용을 크롤링하는 함수"""
    try:
        article_response = requests.get(news_url, headers=headers)
        article_response.raise_for_status()  # HTTP 에러 확인
        article_soup = BeautifulSoup(article_response.text, 'html.parser')

        # 네이버 뉴스 본문 요소 선택 (기사에 따라 선택자가 다를 수 있음)
        content_element = article_soup.select_one('#newsct_article') or article_soup.select_one('#dic_area')

        if content_element:
            return content_element.get_text(strip=True)
        else:
            return "기사 본문 내용을 찾을 수 없습니다."

    except requests.exceptions.RequestException as e:
        print(f"기사 내용 요청 에러 발생: {e}")
        return "기사 내용을 가져오는 데 실패했습니다."
    except Exception as e:
        print(f"기사 내용 파싱 에러 발생: {e}")
        return "기사 내용을 파싱하는 데 실패했습니다."

if __name__ == "__main__":
    # 데이터셋 로드
    df = pd.read_csv('/Users/han-yeeun/mid-project/src/storage/train_data/train-0311.csv', encoding='euc-kr')

    # 수도권 필터링
    metropolitan_areas = ['서울', '경기', '인천']
    metropolitan_df = df[df['공급지역명'].isin(metropolitan_areas)]

    # 수도권 주택명만 추출
    metropolitan_apt = metropolitan_df['주택명'].unique()

    # 결과를 저장할 리스트
    all_news_data = []

    # 각 아파트에 대해 뉴스 크롤링
    for apartment_name in tqdm(metropolitan_apt):
        # 1단계: 검색 URL 생성
        search_url = generate_news_url(apartment_name)
        
        # 2단계: 네이버 뉴스 기사 크롤링 (최대 10개)
        news_data = crawl_naver_news(search_url, max_articles=10)  # 여기를 수정

        # 결과 저장
        for article in news_data:
            article['apartment'] = apartment_name
        all_news_data.extend(news_data)
        
        # 과도한 요청 방지를 위한 대기
        time.sleep(3)

    # 결과를 DataFrame으로 변환
    result_df = pd.DataFrame(all_news_data)

    # CSV 파일로 저장
    result_df.to_csv('수도권_아파트_청약_뉴스_크롤링.csv', index=False, encoding='utf-8-sig')
    print(f"총 {len(result_df)}개의 기사가 크롤링되어 CSV 파일로 저장되었습니다.")

  0%|          | 2/680 [00:34<3:11:21, 16.93s/it]

더 이상 기사 영역이 없습니다.


  1%|          | 5/680 [01:37<3:47:42, 20.24s/it]

더 이상 기사 영역이 없습니다.


  2%|▏         | 14/680 [03:55<3:00:52, 16.30s/it]

...

더 이상 기사 영역이 없습니다.


100%|█████████▉| 677/680 [2:50:19<00:37, 12.52s/it]

더 이상 기사 영역이 없습니다.


100%|██████████| 680/680 [2:51:18<00:00, 15.11s/it]

총 6214개의 기사가 크롤링되어 CSV 파일로 저장되었습니다.


In [6]:
df = pd.read_csv('/Users/han-yeeun/mid-project/src/datasets/수도권_아파트_청약_뉴스_크롤링.csv')
df.head()

,title,content,url,apartment
0,“7인 가구에 무주택 15년 이상”…새해 첫 만점통장 ‘이곳’서 나왔다,의왕 월암지구 대방 디에트르 레이크파크 조감도. [사진 출처 = 대방건설]경기도 의...,https://n.news.naver.com/mnews/article/009/000...,의왕 월암지구 대방 디에트르 레이크파크
1,올해 첫 청약 통장 만점 나온 단지는…서울 아니고 이곳,'의왕 디에트르 레이크파크'서 84점 나와[서울경제]의왕 월암 디에트르 레이크파크 ...,https://n.news.naver.com/mnews/article/011/000...,의왕 월암지구 대방 디에트르 레이크파크
2,"""지금 아니면 내집마련 못한다?""...새해 첫 84점 만점통장 나왔다",아파트 견본주택 전경. 사진=연합뉴스[파이낸셜뉴스] 무적통장으로 불리는 84점 만점...,https://n.news.naver.com/mnews/article/014/000...,의왕 월암지구 대방 디에트르 레이크파크
3,강달러 간다는데 환율 더 오르나 [3분 브리프],[한 눈에 읽기]①금값 내년에도 고공행진 전망②1년새 121% 상승한 비트코인③머스...,https://n.news.naver.com/mnews/article/277/000...,의왕 월암지구 대방 디에트르 레이크파크
4,[1월 1주 분양동향] '검단신도시파밀리에엘리프' 등 5911가구 분양,일반분양 기준으론 5014가구⋯새해 맞아 견본주택 개관은 전무푸른 뱀의 해인 새해 ...,https://n.news.naver.com/mnews/article/031/000...,의왕 월암지구 대방 디에트르 레이크파크


In [7]:
df.tail()

,title,content,url,apartment
6209,"인기 아파트 청약을 통해 살펴본 분양 트렌드, 타워형 경쟁률 낮고 가점...",코로나19 감염 확산이 장기화되면서 국내 경제 산업 전반에 ‘경고음’이 커지고 있다...,https://n.news.naver.com/mnews/article/009/000...,양주옥정유림노르웨이숲
6210,"SM그룹·양우건설, 첫 분양서 미달 `쓴맛`",수도권 분양경기 침체 영향아파트 브랜드 인지도 낮고코로나 여파에 홍보도 못해중견건설...,https://n.news.naver.com/mnews/article/029/000...,양주옥정유림노르웨이숲
6211,“분양경기 최악인데”…마수걸이 분양 나섰다 된서리 맞은 ‘SM·양우건...,"올해 경기도 양주·파주서 첫 분양 나서 두 단지 모두 1순위 미달""SM그룹·양우건설...",https://n.news.naver.com/mnews/article/029/000...,양주옥정유림노르웨이숲
6212,코로나에도…`줍줍`열기는 뜨거웠다,양주 노르웨이숲 무순위 청약주말 아침부터 수백명 몰려14일 양주옥정 노르웨이숲 견본...,https://n.news.naver.com/mnews/article/009/000...,양주옥정유림노르웨이숲
6213,규제 피한 2기 신도시 ‘분양 핫플레이스’ 되나,상반기 14곳 1만3234가구 공급 / 잇단 부동산대책에 비규제 쏠림 가속 / 경기...,https://n.news.naver.com/mnews/article/022/000...,양주옥정유림노르웨이숲
